In [38]:
from google.colab import files
import zipfile
import os

uploaded = files.upload()

zip_name = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall('dataset')

Saving training_set.zip to training_set.zip


In [22]:
import os
import glob
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt


In [23]:
class HCDataset(Dataset):
    def __init__(self, root):
        all_images = sorted([
            f for f in glob.glob(root + "/**/*.png", recursive=True)
            if "Annotation" not in f
        ])

        self.images = []
        for img_path in all_images:
            mask_path = img_path.replace(".png", "_Annotation.png")
            if os.path.exists(mask_path):
                self.images.append(img_path)

        if len(self.images) == 0:
            raise ValueError("No image–annotation pairs found!")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        mask_path = img_path.replace(".png", "_Annotation.png")

        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        img = cv2.resize(img, (32, 32)) / 255.0
        mask = cv2.resize(mask, (32, 32)) / 255.0

        img = torch.tensor(img, dtype=torch.float32).view(-1)
        mask = torch.tensor(mask, dtype=torch.float32).view(-1)

        return img, mask


In [16]:
dataset = HCDataset("/content/dataset")
loader = DataLoader(dataset, batch_size=4, shuffle=True)

print("Samples:", len(dataset))


Samples: 999


In [9]:
class deepLearningModel(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc = nn.Linear(1024, 1024)

    def forward(self, x):
        return self.fc(x)


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

sample_x, _ = next(iter(loader))
input_dim = sample_x.shape[1]

model = deepLearningModel(input_dim).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 10

for epoch in range(epochs):
    total_loss = 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] Loss: {total_loss/len(loader):.4f}")


Epoch [1/10] Loss: 0.0133
Epoch [2/10] Loss: 0.0094
Epoch [3/10] Loss: 0.0081
Epoch [4/10] Loss: 0.0073
Epoch [5/10] Loss: 0.0068
Epoch [6/10] Loss: 0.0065
Epoch [7/10] Loss: 0.0062
Epoch [8/10] Loss: 0.0061
Epoch [9/10] Loss: 0.0059
Epoch [10/10] Loss: 0.0056
